In [1]:
import sys
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.io import mmread
from scipy.io import mmwrite
from scipy.sparse import csr_matrix

# solve the jax and jaxlib issue from https://github.com/google/jax/issues/5501
import cell2location
import scvi

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
import seaborn as sns


Global seed set to 0


In [2]:
results_folder = '../results/visium_axolotl_R12830_resequenced_20220308/cell2location_subtype_out'

# create paths and names to results folders for reference regression and cell2location models
ref_run_name = f'{results_folder}/reference_signatures'
run_name = f'{results_folder}/cell2location_map'


# Start to process snRNA-seq and prepare the gene signature

In [ ]:
original_counts = np.round(csr_matrix(mmread("../data/snRNAseq_countMatrix.mtx")))


In [ ]:
original_counts

In [ ]:
original_meta = pd.read_csv("../data/snRNAseq_countMatrix_metadata.csv", index_col = 0)
original_meta

In [ ]:
original_genes = pd.read_csv("../data/snRNAseq_countMatrix_gene.csv", index_col = 0)
original_genes

In [ ]:
adata_ref = ad.AnnData(original_counts, obs=original_meta, var = original_genes)

In [ ]:
adata_ref
adata_ref.to_df() # double check if the cell barcodes and gene names are correct 

In [ ]:
# filtering 
# pal_data = pal_data[pal_data.obs.cellclusters!="glut_SUBSET_23",]

In [ ]:
from cell2location.utils.filtering import filter_genes
selected = filter_genes(adata_ref, cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12)

# filter the object
adata_ref = adata_ref[:, selected].copy()


In [ ]:
adata_ref

In [ ]:
# prepare anndata for the regression model
cell2location.models.RegressionModel.setup_anndata(adata=adata_ref,
                        # 10X reaction / sample / batch
                        batch_key=None,
                        # cell type, covariate used for constructing signatures
                        labels_key='subtypes',
                        # multiplicative technical effects (platform, 3' vs 5', donor effect)
                        categorical_covariate_keys=['condition']
                       )


In [ ]:
# create the regression model
from cell2location.models import RegressionModel
mod = RegressionModel(adata_ref)

# view anndata_setup as a sanity check
mod.view_anndata_setup()

In [ ]:
# Use all data for training (validation not implemented yet, train_size=1)
# takes ~ 3.5h for 175 epochs, but should increase to 200/250 epochs
mod.train(max_epochs=250, batch_size=2500, train_size=1, lr=0.002, use_gpu=False) 

# plot ELBO loss history during training, removing first 20 epochs from the plot
#mod.plot_history(20)

#mod.train(max_epochs=50, use_gpu=False)


In [ ]:
# plot ELBO loss history during training, removing first 20 epochs from the plot
mod.plot_history(20)


In [ ]:
# In this section, we export the estimated cell abundance (summary of the posterior distribution).
adata_ref = mod.export_posterior(
    adata_ref, sample_kwargs={'num_samples': 1000, 'batch_size': 2500, 'use_gpu': False}
)

# Save model
mod.save(f"{ref_run_name}", overwrite=True)

# Save anndata object with results
adata_file = f"{ref_run_name}/sc_subtypes_v1.h5ad"
adata_ref.write(adata_file)
adata_file

# Reload the process snRNA-seq and model 

In [4]:
# The model and output h5ad can be loaded later like this if needed
#ref_run_name = "/Volumes/groups/tanaka/People/current/jiwang/projects/heart_regeneration/results/visium_axolotl_R12830_resequenced_20220308/cell2location_coarse_out/reference_signatures/"

adata_file = f"{ref_run_name}/sc_subtypes_v1.h5ad"
adata_ref = sc.read_h5ad(adata_file)

OSError: Unable to open file (truncated file: eof = 1047789568, sblock->base_addr = 0, stored_eof = 1048267416)

In [ ]:
mod.plot_QC()


In [ ]:
mod = cell2location.models.RegressionModel.load(f"{ref_run_name}", adata_ref)

In [ ]:
# export estimated expression in each cluster
if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
    inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
else:
    inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_ref.uns['mod']['factor_names']
inf_aver.iloc[0:5, 0:5]

In [ ]:
# load visium data 
vis = sc.read_loom("../data/processed/D1_113_sub_b.loom")


In [ ]:
vis.X = np.round(vis.X)